# Etape 4 : Run Inference

In [ ]:
import torch
print("Torch OK :", torch.__version__)
print("CUDA dispo ?", torch.cuda.is_available())

Torch OK : 2.9.1+cpu
CUDA dispo ? False


In [ ]:
import os

# Se placer à la racine du repo (un niveau au-dessus de notebooks/)
os.chdir("..")
print("Nouveau CWD =", os.getcwd())

Nouveau CWD = c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01


In [ ]:
# Chemins des fichiers
VIDEO_CSV = "data/splits/video_folds_5fold.csv"
FRAMES_ROOT = "data/frames/5_forstep1and2"
SUPERIMG_CSV = "data/splits/super_images_3x3_folds.csv"
SUPERIMG_ROOT = "data/super_images"

# Dossiers des modèles entraînés
FRAMEWISE_DIR = "exp/results/framewise_resnet18"
CONV_DIR = "exp/results/super_images_convnext"
TRES_DIR = "exp/results/super_images_tresnetxl"

print("📁 Chemins chargés :")
for name, val in {
    "VIDEO_CSV": VIDEO_CSV,
    "FRAMES_ROOT": FRAMES_ROOT,
    "SUPERIMG_CSV": SUPERIMG_CSV,
    "SUPERIMG_ROOT": SUPERIMG_ROOT,
    "FRAMEWISE_DIR": FRAMEWISE_DIR,
    "CONV_DIR": CONV_DIR,
    "TRES_DIR": TRES_DIR,
}.items():
    print(f"- {name}: {val}")


📁 Chemins chargés :
- VIDEO_CSV: data/splits/video_folds_5fold.csv
- FRAMES_ROOT: data/frames/5_forstep1and2
- SUPERIMG_CSV: data/splits/super_images_3x3_folds.csv
- SUPERIMG_ROOT: data/super_images
- FRAMEWISE_DIR: exp/results/framewise_resnet18
- CONV_DIR: exp/results/super_images_convnext
- TRES_DIR: exp/results/super_images_tresnetxl


In [ ]:
import subprocess, sys

def run_python(script, args):
    cmd = [sys.executable, script] + args
    print("▶️ Commande :", " ".join(cmd))
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("❌ ERREUR (code", result.returncode, ")")
        print(result.stderr)
    print("\n--- FIN DU SCRIPT ---\n")
    return result

## Inference framewise (5 folds) - Local CPU

In [ ]:
!python src/infer/framewise_infer.py \
  --video_csv data/splits/video_folds_5fold.csv \
  --frames_root data/frames/5_forstep1and2 \
  --ckpt exp/results/framewise_resnet18/resnet18_fold4/best_model.pth \
  --fold 4 \
  --out exp/results/framewise_resnet18/preds_resnet18_fold4.npz

Device: cpu
Fold 4 → 1291 vidéos
Loading checkpoint: exp/results/framewise_resnet18/resnet18_fold4/best_model.pth
Backbone: resnet18
[WARN] Aucune frame trouvée pour la vidéo '25232', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23916_23', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo '26985', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23921_9', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23917_12', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23906_10', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23921_11', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo '25232', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23916_23', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo '26985', utilisatio


Inference:   0%|          | 0/81 [00:00<?, ?it/s]c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)

Inference: 100%|██████████| 81/81 [03:02<00:00,  2.25s/it]


## Inference ConvNeXt (5 folds) - Colab A100

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Clonage du dépôt dans /content (première fois dans cette session)
%cd /content

!git clone https://github.com/Simon-VDC/qv-pipe-classifier.git
%cd qv-pipe-classifier

!ls

/content
Cloning into 'qv-pipe-classifier'...
remote: Enumerating objects: 443, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 443 (delta 139), reused 133 (delta 32), pack-reused 168 (from 1)
Receiving objects: 100% (443/443), 1.21 MiB | 2.56 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/content/qv-pipe-classifier
CONFIG.md      docs		exp	   project_tree.txt  requirements
data	       ENVIRONMENT.md	LICENSE    README.md	     scripts
DATA_NOTES.md  environment.yml	notebooks  reports	     src


In [6]:
!mkdir -p "/content/drive/MyDrive/QV Pipe/exp/results/super_images_convnext"


In [9]:
# Boucle d'inférence ConvNeXt pour les folds 0 à 4
for FOLD in range(5):
    print(f"=============== Fold {FOLD} ===============")

    cmd = f"""
    python -m src.infer.superimages_infer \
        --splits_csv "/content/drive/MyDrive/QV Pipe/data/splits/super_images_3x3_folds_colab.csv" \
        --img_root   "/content/drive/MyDrive/QV Pipe/data/super_images" \
        --ckpt       "/content/drive/MyDrive/QV Pipe/models/super_images_convnext_cb_focal/convnext_base_fold{FOLD}/best_model.pth" \
        --fold       {FOLD} \
        --backbone   "convnext_base" \
        --out        "/content/drive/MyDrive/QV Pipe/exp/results/super_images_convnext/preds_convnext_fold{FOLD}.npz" \
        --batch_size 4 \
        --image_size 1344
    """

    print(cmd)
    !$cmd

=============== Fold 0 ===============

    python -m src.infer.superimages_infer         --splits_csv "/content/drive/MyDrive/QV Pipe/data/splits/super_images_3x3_folds_colab.csv"         --img_root   "/content/drive/MyDrive/QV Pipe/data/super_images"         --ckpt       "/content/drive/MyDrive/QV Pipe/models/super_images_convnext_cb_focal/convnext_base_fold0/best_model.pth"         --fold       0         --backbone   "convnext_base"         --out        "/content/drive/MyDrive/QV Pipe/exp/results/super_images_convnext/preds_convnext_fold0.npz"         --batch_size 4         --image_size 1344
    
Device: cuda
Fold 0 → 578 super-images
Loading checkpoint: /content/drive/MyDrive/QV Pipe/models/super_images_convnext_cb_focal/convnext_base_fold0/best_model.pth
Backbone utilisé : convnext_base
Inference: 100% 145/145 [01:14<00:00,  1.96it/s]

Saved predictions → /content/drive/MyDrive/QV Pipe/exp/results/super_images_convnext/preds_convnext_fold0.npz
=============== Fold 1 ==============

## Inference TResNet-XL (5 folds) - Colab A100

In [10]:
# Création du dossier de sortie (si pas encore existant)
!mkdir -p "/content/drive/MyDrive/QV Pipe/exp/results/super_images_tresnetxl"

# Boucle d'inférence TResNet-XL pour les folds 0 à 4
for FOLD in range(5):
    print(f"=============== Fold {FOLD} (TResNet-XL) ===============")

    cmd = f"""
    python -m src.infer.superimages_infer \
        --splits_csv "/content/drive/MyDrive/QV Pipe/data/splits/super_images_3x3_folds_colab.csv" \
        --img_root   "/content/drive/MyDrive/QV Pipe/data/super_images" \
        --ckpt       "/content/drive/MyDrive/QV Pipe/models/super_images_tresnet_xl_cb_focal_highres/tresnet_xl.miil_in1k_fold{FOLD}/best_model.pth" \
        --fold       {FOLD} \
        --backbone   "tresnet_xl.miil_in1k" \
        --out        "/content/drive/MyDrive/QV Pipe/exp/results/super_images_tresnetxl/preds_tresnetxl_fold{FOLD}.npz" \
        --batch_size 4 \
        --image_size 1344
    """

    print(cmd)
    !$cmd


=============== Fold 0 (TResNet-XL) ===============

    python -m src.infer.superimages_infer         --splits_csv "/content/drive/MyDrive/QV Pipe/data/splits/super_images_3x3_folds_colab.csv"         --img_root   "/content/drive/MyDrive/QV Pipe/data/super_images"         --ckpt       "/content/drive/MyDrive/QV Pipe/models/super_images_tresnet_xl_cb_focal_highres/tresnet_xl.miil_in1k_fold0/best_model.pth"         --fold       0         --backbone   "tresnet_xl.miil_in1k"         --out        "/content/drive/MyDrive/QV Pipe/exp/results/super_images_tresnetxl/preds_tresnetxl_fold0.npz"         --batch_size 4         --image_size 1344
    
Device: cuda
Fold 0 → 578 super-images
Loading checkpoint: /content/drive/MyDrive/QV Pipe/models/super_images_tresnet_xl_cb_focal_highres/tresnet_xl.miil_in1k_fold0/best_model.pth
Backbone utilisé : tresnet_xl.miil_in1k
Inference: 100% 145/145 [00:42<00:00,  3.44it/s]

Saved predictions → /content/drive/MyDrive/QV Pipe/exp/results/super_images_tresnetx

## Ensemble final

In [ ]:
run_python("src/infer/ensemble.py", [])

▶️ Commande : c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\.venv\Scripts\python.exe src/infer/ensemble.py

--- FIN DU SCRIPT ---



CompletedProcess(args=['c:\\Users\\Simon VANDERCOILDEN\\Desktop\\Scolaire\\UTC\\IM05\\TX01\\qv-pipe-classifier\\.venv\\Scripts\\python.exe', 'src/infer/ensemble.py'], returncode=2)

In [ ]:
import os, glob

print("CWD =", os.getcwd())
print("Framewise npz :", glob.glob("exp/results/framewise_resnet18/*.npz"))

CWD = c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\notebooks
Framewise npz : []
